#Relación entre entidades en Wikidata

In [1]:
%pip install qwikidata

In [2]:
from qwikidata.entity import WikidataItem, WikidataProperty, WikidataLexeme 
from qwikidata.linked_data_interface import get_entity_dict_from_api
import time

**En primer lugar obtenemos la información de las entidades que se usarán en esta práctica y las almacenamos en variables para no tener que estar pidiendo una y otra vez las mismas entidades a Wikidata.**

In [3]:
piano = 'Q5994'
piano_dict= get_entity_dict_from_api(piano)
q5994 = WikidataItem(piano_dict)
electronic_keyboard = 'Q1343007'
electronic_keyboard_dict= get_entity_dict_from_api(electronic_keyboard)
q1343007 = WikidataItem(electronic_keyboard_dict)

In [4]:
String_synthesizer = 'Q2355465'
Hans_Zimmer = 'Q76364'
String_synthesizer_dict = get_entity_dict_from_api(String_synthesizer)
Hans_Zimmer_dict = get_entity_dict_from_api(Hans_Zimmer)
q2355465 =  WikidataItem(String_synthesizer_dict)
q76364 = WikidataItem(Hans_Zimmer_dict)

In [5]:
Pirates_of_the_Caribbean= 'Q46717'
Pirates_of_the_Caribbean_dict = get_entity_dict_from_api(Pirates_of_the_Caribbean)
q46717 = WikidataItem(Pirates_of_the_Caribbean_dict)

In [6]:
The_Lion_king = 'Q36479'
The_Lion_king_dict = get_entity_dict_from_api(The_Lion_king)
q36479 =  WikidataItem(The_Lion_king_dict)

In [7]:
Toy_story ='Q171048'
Toy_story_dict = get_entity_dict_from_api(Toy_story)
q171048 = WikidataItem(Toy_story_dict)

In [8]:
Iron_man = 'Q192724'
Iron_man_dict= get_entity_dict_from_api(Iron_man)
q192724 = WikidataItem(Iron_man_dict)

**Creamos un array de tuplas, y cada una de estas tiene el nombre de las propiedades y su identificador en Wikidata.**

In [9]:
properties=[('instance of','P31'),('subclass of','P279'),('has part','P527'),('instrument','P1303'),('genre','P136')]

**Declaramos dos clases: Node y Tree.**
Un nodo puede ser o propiedad o entidad, y el árbol le construiré de tal manera que vayan intercalandose por niveles; es decir, el nivel 0 será una entidad, el nivel 1 tendrán todas las propiedades que se pueden aplicar al padre del nivel 0; el nivel 2 serán todas las entidades que conseguimos al aplicar las propiedades del nivel 1 a la entidad del nivel 0, y asi continuamente.

Así, si lo recorremos en profundidad vamos obteniendo cada camino con la longitud maxima(5 en este caso, por lo que el camino sería en verdad de longitud 10 ya que como hemos dicho, de esos 10 niveles 5 son propiedades y 5 son entidades



In [12]:
class Node:
    def __init__(self, data, tipo, childs = None):
        #print ("Node __init__: " + str(data))
        self.data = data
        self.type = tipo
        self.childs = list()
    def __str__(self):
        if self.type == "property":
            return 'property : ' + str(self.data)
        return 'Entity : ' + str(self.data.get_label())
    
    def _isLeaf(self):
        return len(self.childs) == 0

    def _add(self, new_node):
        # print ("Node _add: " + str(new_node.data) + ' to ' + str(self.data))
        self.childs.append(new_node)

    # print preorder traversal		
    def _preorder(self):
        print (self) 
        for child in self.childs:
            child._preorder()


In [13]:
class Tree:
    def __init__(self,node):
        self.root = node

    def preorder(self):
        print ('----Preorder----')
        self.root._preorder()

Construimos el árbol como había explicado anteriormente.

In [14]:

def get_claims(nodo,profundidad):
    if nodo == None or (profundidad > 4):
        return
    for pr in properties:
        property_node = Node(pr[1],"property")
        claim_group = nodo.data.get_truthy_claim_group(pr[1])
        if len(claim_group) > 0:
            nodo._add(property_node)
            for g in claim_group:
                if g.mainsnak.value_datatype == "wikibase-entityid":
                    entity = g.mainsnak.datavalue.value['id']
                    q_dict = get_entity_dict_from_api(entity)
                    q = WikidataItem(q_dict)
                    entity_node =Node(q,"entidad")
                    property_node._add(entity_node)
                    get_claims(entity_node,profundidad + 1)
            
        

**La siguiente celda de código se encarga de construir un árbol por cada entidad que hemos guardado al principio.La raiz de estos árboles es cada una de las entidades mencionadas.Guardaremos estos arboles en la lista global "list_of_trees"**

La ejecución de la celda puede llevar varios minutos ya que los posibles caminos de longitud 5 son abundantes.

In [15]:
start = time.time()
list_of_trees = list()

tree1 = Tree(Node(q5994,"entidad")) #piano
tree2 = Tree(Node(q1343007,"entidad")) #electronic keyboard
tree3 = Tree(Node(q2355465,"entidad")) #String sintetizer
tree4 = Tree(Node(q76364,"entidad"))#hans Zimmer
tree5 =  Tree(Node(q46717,"entidad"))#Pirates of the Caribbean: The Curse of the Black Pearl
tree6 =  Tree(Node(q36479,"entidad"))#The Lion King
tree7 =  Tree(Node(q171048,"entidad"))#Toy Story
tree8 =  Tree(Node(q192724,"entidad"))#Iron Man

get_claims(tree1.root,0)
get_claims(tree2.root,0)
get_claims(tree3.root,0)
get_claims(tree4.root,0)
get_claims(tree5.root,0)
get_claims(tree6.root,0)
get_claims(tree7.root,0)
get_claims(tree8.root,0)

list_of_trees.append(tree1)
list_of_trees.append(tree2)
list_of_trees.append(tree3)
list_of_trees.append(tree4)
list_of_trees.append(tree5)
list_of_trees.append(tree6)
list_of_trees.append(tree7)
list_of_trees.append(tree8)

print(time.time()-start,"seconds")

597.8447694778442 seconds


**A continuación definiremos algunas funciones auxiliares:**

In [16]:

"""
Recibe una interseccion ( tupla que contiene informacion sobre la intersección) y las lista de intersecciones en la
que iremos almacenando estas.
Recorre las tuplas de la lista de intersecciones buscando si ya existe una interseccion con la misma entidad.
    -Si no existe la añadimos a la lista de intersecciones
    -Si existe y la distancia de la nueva intersección es menor entonces la sustituimos.

"""
def insertarInterseccion(interseccion,lista_intersecciones):
    encontrado = False
    i=0
    max_len = len(lista_intersecciones)
    while i < max_len:
        #si existe una interseccion con esa entidad mirar si esta a menor distancia
        j = 0
        long =len(lista_intersecciones[i]) 
        while not(encontrado) and j < long:
            if lista_intersecciones[i][j][0] == interseccion[0]:
                if(interseccion[1] < lista_intersecciones[i][j][1]):
                    lista_intersecciones[i].clear()
                    lista_intersecciones[i].append(list(interseccion))
                encontrado = True
            j+=1 
            
        if encontrado:
            return
        
        i+=1
    
    #si no la encuentra la añade
    aux = list()
    aux.append(interseccion)
    lista_intersecciones.append(aux)

In [17]:

"""

Esta función se encarga de buscar un nodo en un arbol(pasaremos por parametro el nodo raiz de este), y en caso de
encontrarlo(es decir que existe una intersección), la añadiremos a nuestra lista de intersecciones con la ayuda de la
función anterior.

Las intersecciones serán tuplas que contienen respectivamente :
    -Nombre de la entidad en la que intersectan los caminos
    -Distancia de la relacion(suma de las distancias de la entidad inicial a la de interseccion)
    -Distancia de la entidad inicial a la de intersección del camino del arbol del que seleccionamos los nodos a buscar
    -Distancia de la entidad inicial a la de intersección del camino del arbol que usamos como árbol de búsqueda
    -Camino del primer árbol
    -Camino del segundo árbol
Así tendremos toda la información necesaria de las intersecciones almacenadas

-Parametros:
    -nodo_a_buscar: entidad que estamos buscando en el árbol de busqueda
    -nodo: iremos modificando este parametro de forma que nos sirve para recorrer el árbol de búsqueda
    -profundidad_nodo_buscar
    -profundidad_nodo
    -camino_actual: camino actual hasta el nodo_a_buscar
    -lista_intersecciones
    -camino_busqueda: camino actual hasta el nodo del árbol de búsqueda
    
"""

def getCaminos(nodo_a_buscar,nodo,profundidad_nodo_buscar,profundidad_nodo,camino_actual,lista_intersecciones,camino_busqueda):
    if nodo.type =="entidad" and nodo.data.get_label() == nodo_a_buscar.data.get_label():
        camino_actual.append(nodo)
        camino = list(camino_actual)
        camino_busq = list(camino_busqueda)
        interseccion = (nodo_a_buscar.data.get_label(),int(profundidad_nodo_buscar/2 + profundidad_nodo/2),
                        int(profundidad_nodo_buscar/2),int(profundidad_nodo/2),camino,camino_busq)
        insertarInterseccion(interseccion,lista_intersecciones)
        camino_actual.pop()
        return
        
    elif nodo._isLeaf():
        return
    
    camino_actual.append(nodo)
    for child in nodo.childs:
        getCaminos(nodo_a_buscar,child,profundidad_nodo_buscar,profundidad_nodo + 1,camino_actual,lista_intersecciones,camino_busqueda)
    camino_actual.pop() 

In [18]:

"""
Esta función se encarga de recorrer un árbol e ir entidad por entidad(nodo_a_buscar) buscándola en otro árbol(nodo_arbol_busqueda)
Así,con la ayuda de la función anterior podemos ir nodo por nodo de un árbol buscando si existen intersecciones con 
otro árbol.

-Parámetros:
    -nodo_a_buscar: entidad que buscaremos en el árbol de busqueda. Este puntero se usara para que busquemos todas las
        entidades del árbol.
    -profundidad_nodo_buscar
    -nodo_arbol_busqueda: raiz del árbol de búsqueda
    -lista_intersecciones
    -camino_busqueda:camino actual hasta el nodo del árbol de búsqueda
    
"""
def getRelationsAux(nodo_a_buscar,profundidad_nodo_buscar,nodo_arbol_busqueda,lista_intersecciones,camino_busqueda):
    
    if nodo_a_buscar.type == "entidad":
        camino_actual = list()
        #buscamos intersecciones de esta entidad con el árbol de búsqueda
        getCaminos(nodo_a_buscar,nodo_arbol_busqueda,profundidad_nodo_buscar,0,
                   camino_actual,lista_intersecciones,camino_busqueda)
    
    if nodo_a_buscar._isLeaf():
        return
    
    for child in nodo_a_buscar.childs:
        camino_busqueda.append(child)
        getRelationsAux(child,profundidad_nodo_buscar + 1,nodo_arbol_busqueda,lista_intersecciones,camino_busqueda)
        camino_busqueda.pop()

In [19]:

#Recibe dos árboles y calcula la lista de intersecciones con ayuda de la función anterior
def getRelations(tree1,tree2):
    lista_intersecciones = list()
    camino_busqueda = list()
    camino_busqueda.append(tree1.root)
    getRelationsAux(tree1.root,0,tree2.root,lista_intersecciones,camino_busqueda)
    return lista_intersecciones

In [20]:
#Recibe un camino y lo muestra por pantalla
def caminoToString(camino):
    for nodo in camino:
        if(nodo.type == "property"):
            print(nodo.data +"---->")
        else :
            print(nodo.data.get_label()+ "-->")

In [21]:

#Recibe la lista de intersecciones y la muestra por pantalla
def caminosToString(lista_intersecciones):
    for interseccion in lista_intersecciones:
        for tupla in interseccion:
            print("INTERSECCION EN",tupla[0],"A DISTANCIA",tupla[1],"\n")
            print("==============================================\n")
            print("En C1 la interseccion está a distancia",tupla[3])
            caminoToString(tupla[4])
            print("\n")
            print("En C2 la interseccion está a distancia",tupla[2])
            caminoToString(tupla[5])
            print("\n")
        
    

In [22]:
#Recibe la lista de intersecciones y devuelve las entidades en común
def getEntidadesEnComun(lista_intersecciones):
    entidades = list()
    for interseccion in lista_intersecciones:
        for tupla in interseccion:
            entidades.append(tupla[0])
    return entidades
                

# Caminos mínimos y entidades comunes

Como veremos al calcular las entidades comunes dos a dos,estas tienen muchas entidades en común aunque sean muy genéricas como por ejemplo artificial physical object,product,tool,artificial entity,entity,second-order metaclass', 'third-order metaclass', 'fixed-order metaclass', 'variable-order metaclass', '(meta)class', 'Wikidata metaclass'...
Son terminos que engloban demasiadas entidades.

- piano (Q5994), electronic keyboard (Q1343007),

In [ ]:
comunes1 = getRelations(tree1,tree2)

In [ ]:
len(comunes1)

43

In [ ]:
entidades = getEntidadesEnComun(comunes1)

In [ ]:
print(entidades)

['musical instrument', 'sound generator', 'device', 'equipment', 'artificial physical object', 'product', 'tool', 'converter', 'artificial entity', 'goods', 'entity', 'variable-order metaclass', 'concept', 'observer', 'catalyst', 'musical instrument part', 'work', 'part of', 'class of instruments', 'first-order metaclass', 'second-order metaclass', 'fixed-order metaclass', 'class', 'metaclass', 'keyboard instrument', 'third-order metaclass', '(meta)class', 'Wikidata metaclass', 'formal ontology concept', 'philosophical concept', 'abstract object', 'member of class (philosophy)', 'musical keyboard', 'keyboard', 'input device', 'peripheral equipment', 'user interface', 'receiver', 'input-output device', 'physical interface', 'push-button', 'switch', 'key']


In [ ]:
caminosToString(comunes1)

INTERSECCION EN musical instrument A DISTANCIA 3 


En C1 la interseccion está a distancia 2
electronic keyboard-->
P279---->
keyboard instrument-->
P279---->
musical instrument-->


En C2 la interseccion está a distancia 1
piano-->
P31---->
musical instrument-->


INTERSECCION EN sound generator A DISTANCIA 5 


En C1 la interseccion está a distancia 3
electronic keyboard-->
P279---->
keyboard instrument-->
P279---->
musical instrument-->
P279---->
sound generator-->


En C2 la interseccion está a distancia 2
piano-->
P31---->
musical instrument-->
P279---->
sound generator-->


INTERSECCION EN device A DISTANCIA 6 


En C1 la interseccion está a distancia 3
electronic keyboard-->
P279---->
electronic musical instrument-->
P279---->
electronic instrument-->
P279---->
device-->


En C2 la interseccion está a distancia 3
piano-->
P31---->
musical instrument-->
P279---->
sound generator-->
P279---->
device-->


INTERSECCION EN equipment A DISTANCIA 7 


En C1 la interseccion está a dista

- String synthesizer (Q2355465), Hans Zimmer (Q76364)

In [ ]:
comunes2 = getRelations(tree3,tree4)
len(comunes2)

31

In [ ]:
entidades2 = getEntidadesEnComun(comunes2)
print(entidades2)

['electronic musical instrument', 'class of instruments', 'first-order metaclass', 'second-order metaclass', 'third-order metaclass', 'fixed-order metaclass', 'variable-order metaclass', '(meta)class', 'Wikidata metaclass', 'class', 'metaclass', 'concept', 'formal ontology concept', 'philosophical concept', 'abstract object', 'member of class (philosophy)', 'electrophone', 'musical instrument', 'sound generator', 'device', 'tool', 'equipment', 'converter', 'product', 'musical instrument part', 'work', 'part of', 'electrical apparatus', 'electrical load', 'electronic instrument', 'artificial physical object']


In [ ]:
caminosToString(comunes2)

INTERSECCION EN electronic musical instrument A DISTANCIA 3 


En C1 la interseccion está a distancia 2
Hans Zimmer-->
P1303---->
synthesizer-->
P279---->
electronic musical instrument-->


En C2 la interseccion está a distancia 1
String synthesizer-->
P31---->
electronic musical instrument-->


INTERSECCION EN class of instruments A DISTANCIA 4 


En C1 la interseccion está a distancia 2
Hans Zimmer-->
P1303---->
keyboard instrument-->
P31---->
class of instruments-->


En C2 la interseccion está a distancia 2
String synthesizer-->
P31---->
electronic musical instrument-->
P31---->
class of instruments-->


INTERSECCION EN first-order metaclass A DISTANCIA 6 


En C1 la interseccion está a distancia 3
Hans Zimmer-->
P1303---->
keyboard instrument-->
P31---->
class of instruments-->
P31---->
first-order metaclass-->


En C2 la interseccion está a distancia 3
String synthesizer-->
P31---->
electronic musical instrument-->
P31---->
class of instruments-->
P31---->
first-order metaclass--

- Pirates of the Caribbean: The Curse of the Black Pearl (Q46717),  The Lion King (Q36479)

In [ ]:
comunes3 = getRelations(tree5,tree6)
len(comunes3)

67

In [ ]:
entidades3 = getEntidadesEnComun(comunes3)
print(entidades3)


['film', 'visual artwork', 'work of art', 'creative work', 'intellectual work', 'item of collection or exhibition', 'physical object', 'goods', 'economic concept', 'economics term', 'perceptible object', 'artificial physical object', 'artificial entity', 'entity', 'audiovisual work', 'audiovisual', 'term', 'intangible good', 'series', 'group', 'class', 'metaclass', 'object', 'object of group', 'moving image', 'image', 'depiction', 'motion', 'physical process', 'change', 'linear motion', 'credit', 'acknowledgment', 'trailer', 'advertising', 'economic activity', 'information', 'service', 'film frame', 'photograph', 'motion picture credits', 'inscription', 'text', 'written work', 'communication medium', 'design element', 'title sequence', 'opening credits', 'credit name', 'film genre', 'art genre', 'genre', 'Wikidata metaclass', 'fiction', 'literary genre', 'drama', 'theatrical genre', 'video game theme', 'television genre', 'narrative', 'literature', 'second-order metaclass', 'class or m

In [ ]:
caminosToString(comunes3)

INTERSECCION EN film A DISTANCIA 3 


En C1 la interseccion está a distancia 2
The Lion King-->
P31---->
animated film-->
P279---->
film-->


En C2 la interseccion está a distancia 1
Pirates of the Caribbean: The Curse of the Black Pearl-->
P31---->
film-->


INTERSECCION EN visual artwork A DISTANCIA 5 


En C1 la interseccion está a distancia 3
The Lion King-->
P31---->
animated film-->
P279---->
film-->
P279---->
visual artwork-->


En C2 la interseccion está a distancia 2
Pirates of the Caribbean: The Curse of the Black Pearl-->
P31---->
film-->
P279---->
visual artwork-->


INTERSECCION EN work of art A DISTANCIA 6 


En C1 la interseccion está a distancia 3
The Lion King-->
P136---->
drama-->
P279---->
drama-->
P279---->
work of art-->


En C2 la interseccion está a distancia 3
Pirates of the Caribbean: The Curse of the Black Pearl-->
P31---->
film-->
P279---->
visual artwork-->
P279---->
work of art-->


INTERSECCION EN creative work A DISTANCIA 7 


En C1 la interseccion está a

The Lion King-->
P31---->
animated film-->
P279---->
film-->
P527---->
motion picture credits-->
P279---->
inscription-->
P279---->
communication medium-->


En C2 la interseccion está a distancia 4
Pirates of the Caribbean: The Curse of the Black Pearl-->
P31---->
film-->
P527---->
motion picture credits-->
P279---->
inscription-->
P279---->
communication medium-->


INTERSECCION EN design element A DISTANCIA 9 


En C1 la interseccion está a distancia 5
The Lion King-->
P31---->
animated film-->
P279---->
film-->
P527---->
motion picture credits-->
P279---->
inscription-->
P279---->
design element-->


En C2 la interseccion está a distancia 4
Pirates of the Caribbean: The Curse of the Black Pearl-->
P31---->
film-->
P527---->
motion picture credits-->
P279---->
inscription-->
P279---->
design element-->


INTERSECCION EN title sequence A DISTANCIA 5 


En C1 la interseccion está a distancia 3
The Lion King-->
P31---->
animated film-->
P279---->
film-->
P527---->
title sequence-->


E

- Toy Story (Q171048), Iron Man (Q192724) 

In [ ]:
comunes4 = getRelations(tree7,tree8)
len(comunes4)

56

In [ ]:
entidades4 = getEntidadesEnComun(comunes4)
print(entidades4)

['film', 'visual artwork', 'work of art', 'artificial physical object', 'audiovisual work', 'creative work', 'audiovisual', 'intangible good', 'series', 'group', 'moving image', 'image', 'motion', 'credit', 'trailer', 'advertising', 'film frame', 'photograph', 'motion picture credits', 'inscription', 'title sequence', 'opening credits', 'film genre', 'art genre', 'genre', 'Wikidata metaclass', 'class', 'intellectual work', 'item of collection or exhibition', 'goods', 'physical object', 'artificial entity', 'term', 'object', 'entity', 'object of group', 'depiction', 'physical process', 'change', 'linear motion', 'acknowledgment', 'economic activity', 'information', 'service', 'text', 'communication medium', 'written work', 'design element', 'credit name', 'speculative fiction', 'fiction', 'genre fiction', 'literary genre', 'belletristic literature', 'work', 'speculative fiction film']


In [ ]:
caminosToString(comunes4)

INTERSECCION EN film A DISTANCIA 3 


En C1 la interseccion está a distancia 1
Iron Man-->
P31---->
film-->


En C2 la interseccion está a distancia 2
Toy Story-->
P136---->
buddy film-->
P279---->
film-->


INTERSECCION EN visual artwork A DISTANCIA 5 


En C1 la interseccion está a distancia 2
Iron Man-->
P31---->
film-->
P279---->
visual artwork-->


En C2 la interseccion está a distancia 3
Toy Story-->
P136---->
buddy film-->
P279---->
film-->
P279---->
visual artwork-->


INTERSECCION EN work of art A DISTANCIA 7 


En C1 la interseccion está a distancia 3
Iron Man-->
P31---->
film-->
P279---->
visual artwork-->
P279---->
work of art-->


En C2 la interseccion está a distancia 4
Toy Story-->
P136---->
buddy film-->
P279---->
film-->
P279---->
visual artwork-->
P279---->
work of art-->


INTERSECCION EN artificial physical object A DISTANCIA 7 


En C1 la interseccion está a distancia 3
Iron Man-->
P31---->
film-->
P279---->
visual artwork-->
P279---->
artificial physical object-->

Al estudiar las relaciones entre las películas podemos ver que tienen más entidades en común y algo más concretas, es decir, que engloban a menos entidades demasiado genericas.
En estos casos obtenemos entidades que engloban el arte basicamente, mientras que con las que no eran sobre películas obteniamos clases que podrían englobar tanto un instrumento como un animal.